In [1]:
### parameters
place = 'tel_aviv'
feature = 'road_right'

In [58]:
import os
import zipfile
from glob import glob
import numpy as np


import geopandas as gpd
import pandas as pd
from shapely.ops import unary_union, nearest_points
from shapely.geometry import Point,Polygon,MultiPolygon, LineString

import networkx as nx

# Get the current working directory (e.g., the folder you're running from)
from pathlib import Path
from scipy.ndimage import generic_filter
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'

cwd = Path().resolve()
# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [3]:

# Define paths
input_dir = f'{detail_folder}/raw_data' # Directory containing zip files
output_dir = f'{detail_folder}/unzip_raw_data'  # Directory to store outputs
os.makedirs(output_dir, exist_ok=True)

# Temporary directory for unzipping
temp_dir = os.path.join(output_dir, "unzipped")
os.makedirs(temp_dir, exist_ok=True)

# Initialize list to hold filtered GeoDataFrames
filtered_gdfs = []

In [4]:
cols_to_save = ['oidmigrash', 'msgush', 'msmigrash', 'idtaba', 'tyeudkarka', 'tyeudrashi', 'msshetach', 'geometry']

# Unzip and process each shapefile
for zip_path in glob(os.path.join(input_dir, "*.zip")):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extract_path = os.path.join(temp_dir, os.path.splitext(os.path.basename(zip_path))[0])
        zip_ref.extractall(extract_path)

        for shp_path in glob(os.path.join(extract_path, "**", "*.shp"), recursive=True):
            try:
                gdf = gpd.read_file(shp_path)
                if not gdf.empty:
                    filtered_gdfs.append(gdf)
            except Exception as e:
                print(f"Failed to process {shp_path}: {e}")

# Merge, filter, and save
if filtered_gdfs:
    merged_gdf = gpd.GeoDataFrame(pd.concat(filtered_gdfs, ignore_index=True), crs=filtered_gdfs[0].crs)
    merged_gdf = merged_gdf[cols_to_save]

    merged_gdf.to_file(os.path.join(output_dir, "merged_gdf.shp"))

    # Filter for "תחבורה" in tyeudrashi
    filtered = merged_gdf[merged_gdf["tyeudrashi"] == "תחבורה"]
    filtered.to_file(os.path.join(output_dir, "tyeudrashi.shp"))

    # Remove duplicates by "oidmigrash"
    filtered_unique = filtered.drop_duplicates(subset="oidmigrash")
    final_path = os.path.join(output_dir, "merged_filtered_unique.shp")
    filtered_unique.to_file(final_path)

    print(f"✅ Final shapefile saved to: {final_path}")
else:
    print("⚠️ No matching shapefiles found.")


✅ Final shapefile saved to: C:\Users\achit\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/road_right/unzip_raw_data\merged_filtered_unique.shp


In [78]:
# # Creating of perpendicular lines at specific positions along each street segment — typically at 1/3, 1/2, and 2/3 of the segment — on both the left and right sides.
# 
# ✅ Purpose:
# To sample the cross-section of the street at multiple representative points.
# 
# These perpendiculars simulate measuring from the road center outward toward the edge (e.g., curb or parcel boundary).
# Load your polygon layer
gdf = filtered.copy()
merged_geom = unary_union(gdf.geometry)
# Split multipolygon into separate rows
if isinstance(merged_geom, MultiPolygon):
    geometries = list(merged_geom.geoms)
else:
    geometries = [merged_geom]

# Create a new GeoDataFrame with each part as a separate row
final_gdf = gpd.GeoDataFrame(geometry=geometries, crs=gdf.crs)
final_gdf.to_file(f"{detail_folder}/dissolved_cleaned.shp")
# Convert each polygon to its boundary (LineString or MultiLineString)
gdf_polyline = final_gdf.copy()
gdf_polyline["geometry"] = gdf_polyline["geometry"].boundary
# # Optional: Save to file
gdf_polyline.to_file(f"{detail_folder}/polygon_boundaries_as_lines.shp")
lines_gdf= gpd.read_file(f'{detail_folder}/side_lines_gdf0.shp')

# Extract the start point of each line
lines_gdf["start_point"] = lines_gdf.geometry.apply(lambda geom: Point(geom.coords[0]))

# Convert to GeoDataFrame of points for spatial join
start_points_gdf = gpd.GeoDataFrame(lines_gdf.drop(columns="geometry"), geometry=lines_gdf["start_point"], crs=lines_gdf.crs)

# Spatial join: find start points within road_right polygons
joined = gpd.sjoin(start_points_gdf, final_gdf, how="inner", predicate="within")

# Use index to filter original lines
filtered_lines = lines_gdf.loc[joined.index]


# Save result
filtered_lines.drop(columns=["start_point"]).to_file(f"{detail_folder}/filtered_perpendicular_lines.shp")

In [138]:

def get_line_intersections_with_extension(per_to_test, edge_sindex_temp, gdf_polyline_temp):
    """
    Compute intersections between a line and street boundary polylines.
    If no intersections are found and the line is shorter than `target_length`,
    the line is extended (symmetrically at both ends) to `target_length` and the
    intersection test is repeated.

    Parameters
    ----------
    per_to_test : shapely.geometry.LineString
        The perpendicular line to test.
    edge_sindex_temp : shapely.STRtree or GeoPandas spatial index
        Spatial index built on `gdf_polyline`.
    gdf_polyline_temp : GeoDataFrame
        GeoDataFrame of street boundary polylines.
    target_length : float, optional
        Desired minimum length of the line in the same units as the CRS
        (default is 100).

    Returns
    -------
    intersections : list of shapely.geometry.Point
        List of intersection points (may be empty).
    used_line : shapely.geometry.LineString
        The line geometry that was actually used for the final intersection
        computation (original or extended).
    """
    def _compute_intersections(test_line):
        """Helper: compute all point intersections of test_line with gdf_polyline."""
        candidate_idx = list(edge_sindex_temp.intersection(test_line.bounds))
        candidate_edges = gdf_polyline_temp.iloc[candidate_idx]

        pts = []
        for _, edge_row in candidate_edges.iterrows():
            inter = test_line.intersection(edge_row.geometry)
            if not inter.is_empty:
                if inter.geom_type == "Point":
                    pts.append(inter)
                elif inter.geom_type.startswith("Multi"):
                    pts.extend([g for g in inter.geoms if g.geom_type == "Point"])
        return pts

    # 1. Try original line
    intersections = _compute_intersections(per_to_test)

    if intersections:
        return intersections

    coords = list(per_to_test.coords)
    p0 = Point(coords[0])
    p1 = Point(coords[-1])

    # Direction vector from p0 to p1
    dx = p1.x - p0.x
    dy = p1.y - p0.y


    # Extend both ends along the line direction
    new_p1 = Point(p1.x + dx, p1.y + dy)
    extended_line = LineString([p0, new_p1])


    # Retry intersections with the extended line
    intersections = _compute_intersections(extended_line)
    return intersections



#  Computing the intersection of each perpendicular line with street boundary polylines (converted from polygons), and measuring the distance from the line’s start point to the closest intersection.
# 
# ✅ Purpose:
# To determine the distance from the center of the road to its edge on one side — this gives one half of the street width.
# 
# If done for both left and right sides, you can calculate the full right-of-way width.

# Build spatial index on street edge polylines
edge_sindex = gdf_polyline.sindex

# Prepare list to collect results
results = []

# Process each perpendicular line
for idx, row in filtered_lines.iterrows():
    line = row.geometry
    start_point = row["start_point"]

    intersections= get_line_intersections_with_extension(line, edge_sindex, gdf_polyline)
    # Keep the closest intersection point to the start of the line
    if intersections:
        nearest_point = min(intersections, key=lambda pt: start_point.distance(pt))
        results.append({
            **row.to_dict(),
            "intersection_point": nearest_point,
            "width_side": start_point.distance(nearest_point)
        })

# Create final GeoDataFrame with intersection points
results_gdf = gpd.GeoDataFrame(results, geometry="intersection_point", crs=crs_prj)

# Optional: Save the result
results_gdf.to_file(f"{detail_folder}/efficient_width_side_result.shp")



100.00000000002782
100.00000000003207
100.00000000003207
99.99999999999996
99.99999999999996
99.99999999999996
99.99999999999996
99.99999999999996
100.00000000006678
99.99999999991063
100.00000000005643
99.99999999996079
100.00000000000557
100.00000000002484
99.99999999996513
100.00000000000115
100.00000000000115
100.00000000000115
100.00000000003091
99.99999999998414
100.00000000002734
99.99999999997485
100.00000000001809
100.00000000003587
100.00000000005973
100.00000000003587
100.00000000008941
100.00000000006818
100.00000000000504
100.0000000000005
100.00000000000504
100.00000000001636
99.99999999997439
99.99999999995903
100.00000000009959
100.00000000006602
99.99999999995815
100.00000000006602
100.0000000000328
100.0000000000536
100.0000000000536
100.0000000000536
99.99999999998289
99.99999999997338
99.99999999998289
99.99999999990234
100.00000000006635
99.99999999997922
99.99999999998474
99.99999999999096
99.99999999998474
100.00000000001073
100.0
100.0
100.0
100.0
100.0000000000

In [139]:


def compute_road_right_widths(filtered_lines_in, point_intersections, streets_in,first_loop= True):
    """
    Compute representative left, right, and total road-right widths for each street segment
    by aggregating perpendicular width measurements and merging them with street geometries.

    Parameters
    ----------
    filtered_lines_in : GeoDataFrame

        Perpendicular lines with at least columns ['oidrechov', 'side', ...].
    point_intersections : GeoDataFrame
        Intersection results with columns ['oidrechov', 'side', 'width_side'].
    streets_path : str
        Path to the streets shapefile (must contain 'oidrechov' and 'geometry').
    output_path : str
        Path to save the final street edges shapefile (including filename, e.g. ".../road_right.shp").

    Returns
    -------
    street_edges_gdf : GeoDataFrame
        Street geometries with added fields: 'left', 'right', 'road_right'.
    """

    # Step 1: Merge perpendicular line results (width_side) into filtered_lines by 'oidrechov' and 'side'
    width_df = filtered_lines_in.merge(
        point_intersections[['oidrechov', 'side', 'width_side']],
        on=['oidrechov', 'side'],
        how='left'
    )

    # Step 2: Split 'side' into 'position' and 'onlyside' (e.g., 'start_left' → position='start', onlyside='left')
    width_df[['position', 'onlyside']] = width_df['side'].str.extract(r'(\w+)_(\w+)', expand=True)



    # Step 4: Define function to calculate representative width
    def summarize_width(values):
        values = [v for v in values if pd.notnull(v)]
        if len(values) == 0:
            return np.nan
        elif len(values) == 1:
            return values[0]
        elif len(values) == 2:
            return np.mean(values)
        elif len(values) == 3:
            pairs = [(values[i], values[j]) for i in range(3) for j in range(i + 1, 3)]
            closest_pair = min(pairs, key=lambda pair: abs(pair[0] - pair[1]))
            return np.mean(closest_pair)
        else:
            return np.nan

    # Step 5: Group by street and side, and summarize width
    summary = width_df.groupby(['oidrechov', 'onlyside'])['width_side'].apply(summarize_width).unstack()

    # Step 6: Replace NaNs with 0 for left/right and calculate total width
    summary[["left", "right"]] = summary[["left", "right"]].fillna(0)
    summary["road_right"] = summary["left"] + summary["right"]
    summary.reset_index(inplace=True)

    # Step 7: Merge width estimates into street geometries
    if first_loop:
        street_edges_gdf_in = streets_in.merge(summary, on='oidrechov', how='left')
        street_edges_gdf_in[["road_right"]] = street_edges_gdf_in[["road_right"]].fillna(0)
    else:
        street_edges_gdf_in = streets_in.set_index('oidrechov')
        summary_idx = summary.set_index('oidrechov')[["left", "right", "road_right"]]
        street_edges_gdf_in.loc[summary_idx.index, ["left", "right", "road_right"]] = summary_idx



    return street_edges_gdf_in.reset_index()

# Step 3: Load street geometries
streets = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]
street_edges_gdf= compute_road_right_widths(filtered_lines,results_gdf,streets)
street_edges_gdf.to_file(f'{detail_folder}/{feature}0.shp')




In [140]:
# ---------------------------------------------------------------------------
# Identify perpendicular lines whose start points are NOT within the road-right
# polygons, and keep only those belonging to street segments with road_right = 0
# ---------------------------------------------------------------------------

# 1. Select all start points that are NOT within any polygon in final_gdf
#    We take the original start_points_gdf and remove all indices that appeared
#    in the spatial join (joined). The "~" negates the boolean mask.
not_within = start_points_gdf[~start_points_gdf.index.isin(joined.index)]

# 2. Use the indices of these points to filter the original perpendicular lines.
#    This gives us the full line geometries (not only the points) that are outside.
filtered_lines_not_within = lines_gdf.loc[not_within.index]

# 3. Extract all street segments (oidrechov) that received a calculated
#    road_right value of 0 in the final results. These are problematic segments.
segment_0 = street_edges_gdf[street_edges_gdf["road_right"] == 0]

# 4. Keep only those "not within" lines whose street ID (oidrechov)
#    belongs to a segment with road_right = 0.
#    This helps diagnose only the problematic segments.
not_within = filtered_lines_not_within[
    filtered_lines_not_within['oidrechov'].isin(segment_0['oidrechov'].unique())
]



In [141]:
# ---------------------------------------------------------------------------
# Compute the distance between the first and second intersections of each
# perpendicular line (from `not_within`) with the street boundary polylines.
#
# ✅ Purpose:
# For lines whose start point is NOT within the road-right polygons, we check
# where they intersect the street-edge polylines. If a perpendicular line
# hits TWO edges (two intersections), we take the distance between these two
# intersection points as the street width along that cross-section.
#
# If there is no second intersection (i.e., fewer than 2 intersections),
# we print the corresponding oidrechov for inspection.
# ---------------------------------------------------------------------------

# Build spatial index on street edge polylines
edge_sindex = gdf_polyline.sindex

# Prepare list to collect results
results = []

# Process each perpendicular line in `not_within`
for idx, row in not_within.iterrows():
    line = row.geometry
    start_point = row["start_point"]  # still kept, even if not used in width now

    # Use spatial index to find candidate polylines near the line
    candidate_idx = list(edge_sindex.intersection(line.bounds))
    candidate_edges = gdf_polyline.iloc[candidate_idx]

    # Compute intersections between the perpendicular line and candidate edges
    intersections = []
    for _, edge_row in candidate_edges.iterrows():
        intersection = line.intersection(edge_row.geometry)
        if not intersection.is_empty:
            if intersection.geom_type == "Point":
                intersections.append(intersection)
            elif intersection.geom_type.startswith("Multi"):
                intersections.extend(
                    [geom for geom in intersection.geoms if geom.geom_type == "Point"]
                )

    # We now need AT LEAST TWO intersection points to measure width
    if len(intersections) >= 2:
        # Sort intersections along the line by their position (projection)
        intersections_sorted = sorted(intersections, key=lambda pt: line.project(pt))

        # Take the first and second intersection points along the line
        p1, p2 = intersections_sorted[0], intersections_sorted[1]

        # Width is the distance between these two intersection points
        width_between_edges = p1.distance(p2)

        # Store result; we keep p1 as the geometry (for visualization/diagnostics)
        results.append({
            **row.to_dict(),
            "intersection_point": p1,          # first intersection along the line
            "width_side": width_between_edges  # distance between first and second intersections
        })
# Create final GeoDataFrame with intersection points (first intersection)
results_gdf2 = gpd.GeoDataFrame(results, geometry="intersection_point", crs=crs_prj)

# Optional: Save the result
results_gdf2.to_file(f"{detail_folder}/efficient_width_side_result_not_within.shp")
street_edges_gdf2= compute_road_right_widths(not_within,results_gdf2,street_edges_gdf,False)
street_edges_gdf2['length']= street_edges_gdf2.length
street_edges_gdf2.to_file(f'{detail_folder}/{feature}.shp')

           index                                           geometry  \
oidrechov                                                             
1.0            0  LINESTRING (184322.705 668574.483, 184351.736 ...   
320.0          1  LINESTRING (184251.346 668628.811, 184322.705 ...   
9429.0         2  LINESTRING (184322.705 668574.483, 184297.349 ...   
423.0          3  LINESTRING (184351.736 668559.201, 184381.018 ...   
9427.0         4  LINESTRING (184351.736 668559.201, 184371.851 ...   

                left      right  road_right  
oidrechov                                    
1.0         7.596642  11.572564   19.169206  
320.0       8.520609   9.580508   18.101117  
9429.0      6.253214   3.502499    9.755713  
423.0       4.961748   5.606065   10.567812  
9427.0     13.405233  12.275746   25.680978  
           index                                           geometry  \
oidrechov                                                             
1.0            0  LINESTRING (184322.7

In [135]:
street_edges_gdf2['length']= street_edges_gdf2.length
street_edges_gdf2.to_file(f'{detail_folder}/{feature}.shp')